In [1]:
import numpy as np
import torch, torch.nn as nn, torch.nn.functional as F, torch.optim as optim

In [32]:
train, val, test, data = (np.load("../data/task2_%s.npz" % f)
                    for f in ("train", "val", "test", "data"))
xtr, ytr, xval, yval, xte, yte, xdata, ydata = (
  torch.from_numpy(arr).float()
  for arr in (train["x"], train["y"], val["x"], val["y"], test["x"], test["y"], data["x"], data["y"])
)

# xtr has the format (num households, sequence length, input dim)

In [33]:
class LSTMClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim


        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x.view(len(x), 1, -1))
        out_space = self.fc(lstm_out.view(len(x), -1))
        out_scores = F.log_softmax(out_space, dim=1)
        return out_scores


In [83]:
class RNNClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim


        self.rnn = nn.RNN(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.rnn(x.view(len(x), 1, -1))
        out_space = self.fc(lstm_out.view(len(x), -1))
        out_scores = F.log_softmax(out_space, dim=1)
        return out_scores

In [88]:
num_epochs = 300 #lstm= 1000 --> acc: 0.55, rnn= 300 --> acc: 0.40

#model = LSTMClassifier(input_dim=96, hidden_dim=128, output_dim=10)
model = RNNClassifier(input_dim=96, hidden_dim=128, output_dim=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.3)

x = xtr[0]
x = x[..., None]
y = ytr[0]
y = y.long()

# Train the model
for epoch in range(num_epochs):
    model.zero_grad()
    output_scores = model(x)
    loss = criterion(output_scores, y)
    loss.backward()
    optimizer.step()

    predicted_labels = output_scores.argmax(dim=1)
    correct = 0
    for i in range(len(predicted_labels)):
        if predicted_labels[i] == y[i]:
            correct += 1

    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f, acc: %1.5f" % (epoch, loss.item(), correct/len(predicted_labels)))

Epoch: 0, loss: 2.30090, acc: 0.12089
Epoch: 100, loss: 1.86077, acc: 0.51199
Epoch: 200, loss: 1.34201, acc: 0.60719


In [89]:
x = xte[0]
x = x[..., None]
y = yte[0].long()

with torch.no_grad():
    output_scores = model(x)
    predicted_labels = output_scores.argmax(dim=1)

correct = 0
for i in range(len(predicted_labels)):
    if predicted_labels[i] == y[i]:
        correct += 1

print("accuracy: ", correct/len(predicted_labels))


accuracy:  0.4054794520547945
